In [ ]:
import numpy as np
import os
import torch
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, Subset
from sklearn.model_selection import StratifiedShuffleSplit
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import transforms as T
from torchvision.transforms import ToTensor 
from utils.train import train_model,set_dataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
target_model = torch.load("./models/victims/Cifar10_Resnet18.pt",map_location='cpu')
target_model = target_model.to(device)

In [ ]:
#setting train hyper-parameter
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(target_model.parameters(), lr=0.00005, momentum=0.9, weight_decay=5e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
train_data, test_data, class_num = set_dataset("cifar10")

In [ ]:
if not os.path.exists("./models/attack_model"):
    os.makedirs("./models/attack_model")
for size in [100,500,1000,2500]:
    train_len=len(train_data)

    sss = StratifiedShuffleSplit(n_splits=1, test_size=size, random_state=7)
    indices = list(range(len(train_data)))
    y_test= [y for _, y in train_data]

    for test_index, val_index in sss.split(indices, y_test):
        print(len(val_index), len(test_index))

    val_ds = Subset(train_data, val_index)
    test_ds = Subset(train_data, test_index)
    batch_size = 128
    train_loader = torch.utils.data.DataLoader(
                     dataset=val_ds,
                     batch_size=batch_size,
                     shuffle=True)
    test_loader = torch.utils.data.DataLoader(
                    dataset=test_data,
                    batch_size=batch_size,
                    shuffle=True)
   
    model = train_model(target_model, criterion, optimizer_ft, exp_lr_scheduler,device, train_loader, test_loader, 10)
    torch.save(model,'./models/attack_model/Finetuning_'+str(size)+'.pt')